Check for duplicate PIDs in the dataset

In [23]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('../data/w1w2w3w4w5_indices_weights_jul12_2022.csv', low_memory=False)

# Check for duplicate PIDs
duplicate_pids = df[df.duplicated('PID', keep=False)]

if not duplicate_pids.empty:
    print("There are duplicate PIDs in the dataset.")
    duplicate_pids.to_csv('../data/duplicate_pids.csv', index=False)
    print("Duplicate PIDs have been saved to '../data/duplicate_pids.csv'.")
else:
    print("All PIDs are unique.")

There are duplicate PIDs in the dataset.
Duplicate PIDs have been saved to '../data/duplicate_pids.csv'.


---
Filter Wave Survey Data

In [24]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("../data/duplicate_pids.csv", low_memory=False)

# Identify valid sequences (rolling window approach)
df['valid_seq'] = df['WAVE'].rolling(3).apply(lambda x: list(x) == [2,3,4], raw=True)

# Flag rows belonging to valid sequences
df['keep'] = df['valid_seq'].shift(-2).fillna(0).astype(bool) | df['valid_seq'].shift(-1).fillna(0).astype(bool) | df['valid_seq'].fillna(0).astype(bool)

# Filter the rows
filtered_df = df[df['keep']].drop(columns=['valid_seq', 'keep'])

# Save to a new CSV file
filtered_df.to_csv("../data/filtered_duplicate_pids.csv", index=False)

print("Filtered data saved to filtered_duplicate_pids.csv")

Filtered data saved to filtered_duplicate_pids.csv


---
Likert Scale Standardization

In [25]:
import pandas as pd

# Load the CSV file from the data directory
df = pd.read_csv('../data/filtered_duplicate_pids.csv', low_memory=False)

# Some variables use a five-point Likert scale (1 to 5), while others use a four-point scale (1 to 4). 
# Since some scales are missing a neutral category (like 3), we need to standardize them.
# I chose to rescale the 1-4 scale to match the 1-5 scaleby applying a linear transformation:
# 1 (Not at all)         → 1
# 2 (Only a little)      → 2.33
# 3 (A moderate amount)  → 3.67
# 4 (A great deal)       → 5
mapping = {1: 1, 2: 2.33, 3: 3.67, 4: 5}

# List of variables to be mapped
variables_to_map = ['cc4_world', 'cc4_wealthUS', 'cc4_poorUS', 'cc4_comm', 'cc4_famheal', 'cc4_famecon']

# Apply the mapping to the specified variables
for var in variables_to_map:
    df[var] = df[var].map(mapping)

# Save the cleaned data to a new CSV file
df.to_csv('../data/likert_scale.csv', index=False)

print("Data has been standardized and saved to 'data/likert_scale.csv'.")

Data has been standardized and saved to 'data/likert_scale.csv'.


---
Fill missing values

In [26]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from skimpy import skim  # Equivalent to skimr::skim() in R

# Load the dataset
df = pd.read_csv('../data/likert_scale.csv', low_memory=False)

# Define ccSolve columns in decreasing order of WTP
ccSolve_columns = ["ccSolve100", "ccSolve50", "ccSolve10", "ccSolve1", "ccSolve0"]
cc_policy_columns = ["cc_pol_tax", "cc_pol_car"]

# Step 1: Fill downward (Lower WTP should be at least as approved as higher WTP)
for i in range(len(ccSolve_columns) - 1):  
    higher_col = ccSolve_columns[i]
    lower_col = ccSolve_columns[i + 1]

    # If lower response is missing but higher response exists, fill with gradual increase
    df[lower_col] = df[lower_col].fillna(df[higher_col] + 0.75)

# Step 2: Fill upward (Higher WTP should be less approved than lower WTP)
for i in range(len(ccSolve_columns) - 1, 0, -1):  
    lower_col = ccSolve_columns[i]
    higher_col = ccSolve_columns[i - 1]

    # If higher response is missing but lower response exists, fill with gradual decrease
    df[higher_col] = df[higher_col].fillna(df[lower_col] - 1.5)

# Step 3: Handle respondents with all missing `ccSolve` responses
# If all `ccSolve` values are missing for a respondent, fill with a neutral value (3)
df[ccSolve_columns] = df[ccSolve_columns].apply(lambda row: row.fillna(3) if row.isna().all() else row, axis=1)

# Step 4: Ensure values stay within 1-5
df[ccSolve_columns] = df[ccSolve_columns].clip(1, 5)

# Step 5: Fill missing values for cc_pol_tax and cc_pol_car with 3 (Neutral)
for cc_policy in cc_policy_columns:
    df[cc_policy] = df[cc_policy].fillna(3)

# Save the filled dataset
df.to_csv("../data/cleaned_data.csv", index=False)